<a href="https://colab.research.google.com/github/PGLJIJI/data-course-sample/blob/main/Recommendation_System_Rule_Based_PGL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import gzip, json
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-26 12:32:29--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  18.7MB/s    in 0.8s    

2021-12-26 12:32:30 (18.7 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2021-12-26 12:32:30--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  14.1MB/s    in 0.7s    

202

In [3]:
## Testing Data
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

In [5]:
ratings

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1407628800,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200,2011-12-25
...,...,...,...,...,...
371340,B01HJEGTYK,A202DCI7TV1022,1.0,1500508800,2017-07-20
371341,B01HJEGTYK,A3FSOR5IJOFIBE,5.0,1489622400,2017-03-16
371342,B01HJEGTYK,A1B5DK6CTP2P24,5.0,1488326400,2017-03-01
371343,B01HJEGTYK,A23OUYS5IRMJS9,2.0,1487635200,2017-02-21


In [6]:
metadata

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32887,[],,[],,"Barielle Pro Textured Grip Cuticle Nipper, Purple",[],,,[],"2,145,325 in Beauty & Personal Care (",[],{'ASIN: ': 'B01HIWLLUK'},All Beauty,,,$9.95,B01HIWLLUK,[],[]
32888,[],,[],,(Buy 3 Get 1 Free) Salon Perfect Eye Makeup Co...,[],,Salon Perfect,[],"1,639,713 in Beauty & Personal Care (",[],"{'ASIN: ': 'B01HJ1K3YK', 'UPC:': '671635851871'}",All Beauty,,,,B01HJ1K3YK,[],[]
32889,[],,[],,NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...,"[B01KON9B4S, B079X3YFXS, B00M79OYS6, B000JN4CR...",,,[],"207,410 in Beauty & Personal Care (","[B01KON9B4S, B000JN4CR0, B071ZHMRHS, B01HJ84TN...","{'Shipping Weight:': '1 pounds (', 'ASIN: ': '...",All Beauty,,,$55.63,B01HJ84SGM,[],[]
32890,[],,[Brand new and high quality<br> Enables fast v...,,12 White Feather Shuttlecocks Birdies Badminto...,[],,GBSTORE,[],"965,673 in Beauty & Personal Care (",[],"{'Shipping Weight:': '4.8 ounces (', 'ASIN: ':...",All Beauty,,,$12.99,B01HJASD20,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [7]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]

In [8]:
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [9]:
ratings_testings_by_user

{'A100XQFWKQ30O2': ['B01DKQAXC0'],
 'A103T1QOGFCSEH': ['B01DKQAXC0'],
 'A106UKKSJ2KXPF': ['B01ENS6XDC'],
 'A10A7GV4D5A11V': ['B01BFSNF66'],
 'A1119JJ37ZLB8R': ['B01CJNZKZK'],
 'A113UOOLBSZN52': ['B01EKY0M22'],
 'A12M4U7WK4ALCR': ['B01DKQAXC0'],
 'A12T8YTW6VWT7S': ['B01G53GFKO'],
 'A1364JXGKB46MM': ['B01GLA54SA'],
 'A137DALOQFKBTI': ['B01AWXGD3M'],
 'A13FEZ3WV7S2EY': ['B01BHN3EHE'],
 'A13IV4I1B0RXMG': ['B01DKQAXC0'],
 'A13JU88JAHN72I': ['B017I6B6GK'],
 'A13K55R6VH1OOD': ['B01FNJ9MOW'],
 'A13P7VFU075A': ['B01H3ZQ2NI'],
 'A13SWYE4QLB6NG': ['B01DVLHVPQ'],
 'A13ZTQ0Q4ATA41': ['B01AE1TJV0'],
 'A142EDN04OD62U': ['B01CW24JXC'],
 'A142I22FIC8MZK': ['B01CD7JK9E'],
 'A14834QTII5TLT': ['B01GK8P0CS'],
 'A14A447VPACTBC': ['B01B3R5EFO'],
 'A14AP6MN5XO6LB': ['B01E7UKR38'],
 'A14CLF25IX25US': ['B01DLR9IDI'],
 'A14LYXC3HTBAHI': ['B01FT6TMQM'],
 'A14VUW4KZ34EOE': ['B01DKQAXC0'],
 'A14Y32P26G9YL': ['B018WCT01C'],
 'A157T25PBS7MX4': ['B01CX5KCIE'],
 'A15HZDSERD85C8': ['B01DDWA5II'],
 'A15JJ8J1FGADIX': ['B0

In [10]:
metadata['rank'] = metadata['rank'].str.split(expand = True)[0] #取出 rank 中的數字部分
metadata['rank'] = metadata['rank'].str.replace(',', '')
metadata['rank'] = pd.to_numeric(metadata['rank'])

In [11]:
metadata[['rank']].describe()

,rank
count,3.245200e+04
mean,1.288190e+06
std,8.151714e+05
min,3.500000e+01
25%,6.775218e+05
50%,1.133461e+06
75%,1.751104e+06
max,9.549407e+06


In [ ]:
metadata[['rank']].quantile(0.1)

rank    342638.7
Name: 0.1, dtype: float64

In [12]:
item_ratings = ratings_trainings[ (ratings_trainings['DATE'] >= '2018-08-01') ].groupby('asin')[['overall']].agg(['mean', 'count']).reset_index()
item_ratings.sort_values(by=('overall', 'count'))

asin   overall      
                     mean count
0    B000MAJD4W  5.000000     1
496  B01B6YTFRC  5.000000     1
498  B01BCCRJO4  5.000000     1
499  B01BE77YOC  5.000000     1
500  B01BGG1RNK  5.000000     1
..          ...       ...   ...
256  B012Z7IHHI  3.230769    26
643  B01DLR9IDI  5.000000    26
271  B013XKHA4M  4.517241    29
161  B00W259T7G  4.113208    53
641  B01DKQAXC0  3.920000    75

[826 rows x 3 columns]

In [13]:
item_ratings.describe(percentiles=[.75,.8, .9])

overall            
             mean       count
count  826.000000  826.000000
mean     3.973179    2.012107
std      1.390220    4.047877
min      1.000000    1.000000
50%      5.000000    1.000000
75%      5.000000    2.000000
80%      5.000000    2.000000
90%      5.000000    3.000000
max      5.000000   75.000000

In [14]:
item_ratings[('overall', 'count')]


0      1
1      1
2      1
3      1
4      1
      ..
821    1
822    1
823    1
824    2
825    1
Name: (overall, count), Length: 826, dtype: int64

In [15]:
# long & short tail
def if_hot_selling(rank):
  if rank <= 342638:
    return 'short_tail'
  else:
    return 'long_tail'
metadata['if_hot_selling'] = metadata['rank'].apply(lambda x: if_hot_selling(x))

item_if_hot_selling = metadata[['asin', 'if_hot_selling']]
item_if_hot_selling

,asin,if_hot_selling
0,6546546450,long_tail
1,7178680776,long_tail
2,7250468162,long_tail
3,7367905066,long_tail
4,7414204790,short_tail
...,...,...
32887,B01HIWLLUK,long_tail
32888,B01HJ1K3YK,long_tail
32889,B01HJ84SGM,short_tail
32890,B01HJASD20,long_tail


In [16]:
user_item_rating_avg = ratings_trainings.groupby(['reviewerID', 'asin'])[['overall']].mean().reset_index()
user_item_rating_avg

,reviewerID,asin,overall
0,A0010876CNE3ILIM9HV0,B01FHDYGQ8,5.0
1,A001170867ZBE9FORRQL,B01B18T01Y,5.0
2,A001170867ZBE9FORRQL,B01FCW88D6,5.0
3,A0028738FYF1SKPPC7B1,B00TZ8XK1E,5.0
4,A0045356A23634W7RI4M,B018H0EJI8,3.0
...,...,...,...
361007,AZZXKG9AGRVGU,B01DVW54I6,5.0
361008,AZZYW4YOE1B6E,B003KWO3UI,5.0
361009,AZZZ5UJWUVCYZ,B00IIZG80U,5.0
361010,AZZZ5UJWUVCYZ,B01CZC20DU,5.0


In [17]:
item_also_buy = metadata[['asin', 'also_buy']]

In [18]:
item_also_view = metadata[['asin', 'also_view']]

In [19]:
metadata[['brand', 'asin']].groupby('brand').count().sort_values(by='asin')

,asin
brand,
Leggings,1
Nigella Therapy,1
Nicorobin,1
Nicole Miller,1
Nicki Minaj Minajesty,1
...,...
Gillette,83
Philips Norelco,88
L'Oreal Paris,113


In [20]:
brand_item_df = metadata[['brand', 'asin']].groupby('brand').agg({'asin': lambda x: x.tolist()}).reset_index()# key=brand , values=asin
brand_item_df

,brand,asin
0,,"[7178680776, 7367905066, 7538626107, 898580208..."
1,#Flashmob,[B00ROFDXVW]
2,#Healthy,"[B00DXZXNG2, B00DYQ2CU8]"
3,#R5,[B002O4LLGM]
4,(3 Pack) NYC Ultra Moist Lipwear - Violet Shin...,[B00OTZ2NES]
...,...,...
7858,younique,[B014JO6U1A]
7859,youthwaters,[B01H2DXMAC]
7860,yoyomax,[B00K0BLTTU]
7861,zila,[B00125NS9M]


In [22]:
brand_item_df

,brand,asin
0,,"[7178680776, 7367905066, 7538626107, 898580208..."
1,#Flashmob,[B00ROFDXVW]
2,#Healthy,"[B00DXZXNG2, B00DYQ2CU8]"
3,#R5,[B002O4LLGM]
4,(3 Pack) NYC Ultra Moist Lipwear - Violet Shin...,[B00OTZ2NES]
...,...,...
7858,younique,[B014JO6U1A]
7859,youthwaters,[B01H2DXMAC]
7860,yoyomax,[B00K0BLTTU]
7861,zila,[B00125NS9M]


In [27]:
user_brand_df = ratings_trainings.merge(metadata[['asin', 'brand']]).groupby('reviewerID').agg({'brand': lambda x: x.tolist()}).reset_index()
user_brand_df

,reviewerID,brand
0,A0010876CNE3ILIM9HV0,[illunt]
1,A001170867ZBE9FORRQL,"[Wazor, ]"
2,A0028738FYF1SKPPC7B1,[Indian Earth]
3,A0045356A23634W7RI4M,[GENERIC]
4,A006277084SDH2LTUV6X,[]
...,...,...
318781,AZZW8CF93X1TC,[Technic]
318782,AZZWNU4RB5BY1,[BAISIDAI]
318783,AZZXKG9AGRVGU,[]
318784,AZZYW4YOE1B6E,[Gillette]


In [23]:
short_tail_item = item_if_hot_selling[item_if_hot_selling['if_hot_selling'] == 'short_tail']
long_tail_item = item_if_hot_selling[item_if_hot_selling['if_hot_selling'] == 'long_tail']

In [24]:
short_tail_item

,asin,if_hot_selling
4,7414204790,short_tail
20,9790787006,short_tail
26,B000050B6B,short_tail
35,B000050FDY,short_tail
37,B000052YAN,short_tail
...,...,...
32872,B01HI1YKIG,short_tail
32873,B01HI5VPW6,short_tail
32884,B01HIPOQ2M,short_tail
32886,B01HIWKGOM,short_tail


In [33]:
from tqdm import tqdm
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    

    for user in tqdm(users):
      # Bought & overall(review) high (4~5)
      user_rating = user_item_rating_avg[user_item_rating_avg['reviewerID'] == user]
      user_rating_high = user_rating[user_rating['overall'] >= 4.5]
      item_condition_1 = user_rating_high['asin'].tolist()
      

        # also buy
      item_condition_1_1 = item_also_buy[item_also_buy['asin'].isin(item_condition_1)]['asin'].tolist()
        # also view
      item_condition_1_2 = item_also_view[item_also_view['asin'].isin(item_condition_1)]['asin'].tolist()


      # Not Bought & overall(review) high (4~5) last 30 days
      item_condition_2_1 = item_ratings[item_ratings[('overall', 'mean')] >= 5.]['asin'].tolist()

      # Review count last 30 days
      item_condition_2_2 = item_ratings[item_ratings[('overall', 'count')] >= 16.]['asin'].tolist()

      # Brand item list
      
      user_bought_brand_list = user_brand_df[user_brand_df['reviewerID'] == user]['brand'].tolist()
      item_condition_3 = brand_item_df[brand_item_df['brand'].isin(user_bought_brand_list)]['asin'].tolist()

      # print(user)

      # short tail 
      import random 
      def select_candidate(item_df, condition_list,  count):
        answer_all = []
        for condition in condition_list:
          answer = item_df[item_df['asin'].isin(condition)]['asin'].tolist()
          if len(answer) >= count:
            answer = random.choices(answer, k=count)

          if len(answer_all) <= count:
            unique_answer = list(set(answer))
            answer_all.extend(unique_answer)
          else:
            return list(set(answer_all))[:count]

        return list(set(answer_all))[:count]
        # return []

        #   answer_all.extend(answer)

        # return random.choices(list(set(answer_all)), k=count)

      short_count = int(round(k*.4))
      long_count = k - short_count

      candidate_list = [item_condition_1_1, item_condition_1_2, item_condition_1, item_condition_3, item_condition_2_2, item_condition_2_1]
      
      short_answer = select_candidate(item_df=short_tail_item, condition_list=candidate_list, count=short_count)
      long_answer = select_candidate(item_df=long_tail_item, condition_list=candidate_list, count=long_count)

      recommendations.update(
              {user: short_answer + long_answer}
      )

    return recommendations
ratings_by_user = recommender(ratings_trainings, users)
# ratings_by_user

100%|██████████| 584/584 [01:23<00:00,  7.03it/s]


In [29]:
ratings_by_user

{'A100XQFWKQ30O2': ['B00O3097YI',
  'B015QBPIY0',
  'B01DKQAXC0',
  'B018ZLH1NY',
  'B0115S8TUI',
  'B016QLPERU',
  'B0139LLU9G',
  'B01GR2V8Z0',
  'B01FQQMNLQ',
  'B015HVN7II'],
 'A103T1QOGFCSEH': ['B01AVJCDYA',
  'B012Z7IHHI',
  'B01BYGXEZG',
  'B015N6GYR8',
  'B00QF207JQ',
  'B00KPYZ3D0',
  'B01A0TPVDG',
  'B01E61C2FE',
  'B0141CMAAA',
  'B01DLR9IDI'],
 'A106UKKSJ2KXPF': ['B01AVJCDYA',
  'B012Z7IHHI',
  'B01E0TOXC2',
  'B01G0NBO9G',
  'B016QLPERU',
  'B00WJ6SZX2',
  'B00U9Q4QXM',
  'B01ED6KKP6',
  'B01BTTA6HW',
  'B01D5LPRC6'],
 'A10A7GV4D5A11V': ['B01AVJCDYA',
  'B01BN7F82S',
  'B00W259T7G',
  'B01EAWILH2',
  'B017A746FU',
  'B01GNMXI66',
  'B00T0SXWLM',
  'B00NWUF8Q6',
  'B0106WRZV8',
  'B01DLR9IDI'],
 'A1119JJ37ZLB8R': ['B013XKHA4M',
  'B012Z7IHHI',
  'B01AVJCDYA',
  'B00O53GQK6',
  'B0167JTCOM',
  'B00WJ6SZX2',
  'B01AZXACKO',
  'B01DEBE69C',
  'B0178LU0Y4',
  'B01DLR9IDI'],
 'A113UOOLBSZN52': ['B00W259T7G',
  'B00R1SVCS8',
  'B01CSVKEYK',
  'B01DKQAXC0',
  'B01BMBMN1Y',
  'B01B

In [ ]:
answer = short_tail_item[short_tail_item['asin'].isin(item_condition_2)]['asin'].tolist()
list(set(answer))

In [34]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))
    score = total / len(ratings_testings)
    return score
evaluate(ratings_testings_by_user, ratings_by_user)

0.030508474576271188